In [0]:
%sql
CREATE TABLE bits_pilani.raw_sch.campaign_templates (
  campaign_key STRING,
  campaign_type STRING,
  offer_template STRING,
  tone STRING,
  call_to_action STRING
);

INSERT INTO bits_pilani.raw_sch.campaign_templates VALUES
  ("loyalty_reward", "Loyalty Reward", "Exclusive reward: $20 off for being a loyal guest", "Warm and appreciative", "Claim Your Reward"),
  ("seasonal_sale", "Seasonal Sale", "25% off everything in {top_category}", "Energetic and persuasive", "Grab the Deal Now"),
  ("reactivation", "Reactivation", "Come back for 15% off your next purchase", "Empathetic and inviting", "Return & Save"),
  ("new_arrival", "New Arrivals", "Explore our latest in {top_category}", "Friendly and curious", "Shop New Arrivals");


In [0]:
campaign_df = spark.table("bits_pilani.raw_sch.campaign_templates")
campaign_dict = {
    row["campaign_key"]: {
        "campaign_type": row["campaign_type"],
        "offer": row["offer_template"],
        "tone": row["tone"],
        "cta": row["call_to_action"]
    }
    for row in campaign_df.collect()
}


In [0]:
%sql
CREATE OR REPLACE TABLE bits_pilani.gold_sch.guest_persona_campaign_enriched AS
SELECT *,
  CASE
    WHEN days_since_last_txn > 90 THEN 'reactivation'
    WHEN total_transactions > 10 THEN 'loyalty_reward'
    WHEN avg_discount_pct > 8 THEN 'seasonal_sale'
    ELSE 'new_arrival'
  END AS campaign_key
FROM bits_pilani.gold_sch.guest_persona_enriched;
